In [23]:
import numpy as np
import tensorflow as tf
from sklearn import preprocessing as p
import pandas as pd

In [29]:
data_source = "https://archive.ics.uci.edu/ml/machine-learning-databases/restricted/breast-cancer/breast-cancer.data"

names = ["class", "age", "menopause", "tumor-size", "inv-nodes", "node-caps", "deg-malig", "breast", "breast-quad", "irradiat"]

df = pd.read_csv(data_source, names=names)
df.dtypes

class          object
age            object
menopause      object
tumor-size     object
inv-nodes      object
node-caps      object
deg-malig       int64
breast         object
breast-quad    object
irradiat       object
dtype: object

In [30]:
df[:5]

,class,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat
0,no-recurrence-events,30-39,premeno,30-34,0-2,no,3,left,left_low,no
1,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,right,right_up,no
2,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,left,left_low,no
3,no-recurrence-events,60-69,ge40,15-19,0-2,no,2,right,left_up,no
4,no-recurrence-events,40-49,premeno,0-4,0-2,no,2,right,right_low,no


In [31]:
for col in ["class", "age", "menopause", "tumor-size", "inv-nodes", "node-caps", "breast", "breast-quad", "irradiat"]:
    df[col] = df[col].astype('category')
df.dtypes

class          category
age            category
menopause      category
tumor-size     category
inv-nodes      category
node-caps      category
deg-malig         int64
breast         category
breast-quad    category
irradiat       category
dtype: object

In [32]:
df= df.dropna()

In [33]:
y_data = df.pop("class")
y_data = pd.get_dummies(y_data).as_matrix()

x_data = df
x_data.insert(0, "bias", 1)
x_data = pd.get_dummies(x_data).as_matrix()


In [34]:
def train_navie_bayes_possiblities(x_data, y_data):
    num_total_case = len(x_data)
    num_attributes = len(x_data[0])
    num_class_series =  np.unique(y_data, return_counts=True)
    num_class_possbilities = num_class_series[1] / num_total_case
    
    case_indexes = []
    for case in num_class_series[0]:
        case_indexes.append(np.where(y_data == case)[0])
        
    possbiliteis_vector = []
    for i, case_index in enumerate(case_indexes):
        possbiliteis_vector.append( (np.sum(x_data[case_index], axis=0)+1  ) / (num_class_series[1][i]+2) )
    return num_class_possbilities, possbiliteis_vector

def infsum(data, **args):
    return np.ma.filled(np.ma.masked_array(data,np.isinf(data)).sum(**args), fill_value=np.nan)


In [35]:
from sklearn.cross_validation import KFold
kf = KFold(len(y_data), n_folds=2)

for train_index, test_index in kf:
    x_train, x_test = x_data[train_index], x_data[test_index]
    y_train, y_test = y_data[train_index], y_data[test_index]
    
    num_class_possbilities, possbiliteis_vector = train_navie_bayes_possiblities(x_train,y_train)

    expected_possbilites = []
 
    for i, case in enumerate(num_class_possbilities): 
        sum_vector = np.log(x_test * possbiliteis_vector[i])
        sum_possbilities = infsum(sum_vector, axis=1)
        expected_possbilites.append(sum_possbilities)
    
    clasfiy_vector = np.argmax(np.array(expected_possbilites), axis=0)
    print(sum(clasfiy_vector == y_data[test_index].ravel())/ len(y_data[test_index].ravel()))


/home/seong_cheol_kim/miniconda3/envs/ml_python/lib/python3.4/site-packages/ipykernel/__main__.py:18: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


TypeError: 'bool' object is not iterable

In [81]:
from sklearn import linear_model, datasets


for train_index, test_index in kf:
    x_train, x_test = x_data[train_index], x_data[test_index]
    y_train, y_test = y_data[train_index], y_data[test_index]

    logreg = linear_model.LogisticRegression(C=1e5)
    logreg.fit(x_train, y_train.ravel())
    
    print(sum(logreg.predict(x_test) == y_test.ravel()) / y_data.shape[0])

0.5


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 1.0

In [85]:
from sklearn.naive_bayes import GaussianNB

for train_index, test_index in cv :
    x_train, x_test = x_data[train_index], x_data[test_index]
    y_train, y_test = y_data[train_index], y_data[test_index]
    
    clf = GaussianNB()
    clf.fit(x_train, y_train)
    result = clf.predict(x_test)
    
    print(sum(result == y_test) / len(y_test))

[ 0.63953488  0.63953488  0.63953488  0.36046512  0.63953488  0.63953488
  0.63953488  0.36046512  0.63953488  0.63953488  0.36046512  0.63953488
  0.63953488  0.63953488  0.63953488  0.63953488  0.36046512  0.63953488
  0.36046512  0.63953488  0.63953488  0.36046512  0.63953488  0.36046512
  0.63953488  0.63953488  0.36046512  0.36046512  0.63953488  0.36046512
  0.36046512  0.63953488  0.63953488  0.63953488  0.63953488  0.36046512
  0.63953488  0.63953488  0.63953488  0.63953488  0.63953488  0.63953488
  0.63953488  0.36046512  0.36046512  0.63953488  0.63953488  0.36046512
  0.63953488  0.36046512  0.63953488  0.36046512  0.36046512  0.36046512
  0.63953488  0.63953488  0.36046512  0.63953488  0.63953488  0.36046512
  0.36046512  0.63953488  0.36046512  0.63953488  0.63953488  0.63953488
  0.36046512  0.63953488  0.36046512  0.63953488  0.63953488  0.63953488
  0.36046512  0.63953488  0.36046512  0.36046512  0.63953488  0.36046512
  0.63953488  0.63953488  0.36046512  0.63953488  0

/home/leejaeho/miniconda3/envs/ml_python/lib/python3.4/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/leejaeho/miniconda3/envs/ml_python/lib/python3.4/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/leejaeho/miniconda3/envs/ml_python/lib/python3.4/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/leejaeho/miniconda3/envs/ml_python/lib/python3.4/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A 